
# Start — Data Package Selector (JupyterLite)

Pick a data package:

- **Use sample** (built-in tiny sample; no download), or
- **Download** a package (zip) into local browser storage (IndexedDB).

> This runs fully in your browser (Pyodide). First run may take a few seconds as Python loads.


In [ ]:

# Install widgets (pure-Python; fine in Pyodide)
import micropip
await micropip.install("ipywidgets==8.1.1")


In [ ]:

# Selector UI: download external package into browser storage (IndexedDB)
import io, os, zipfile, shutil, asyncio
from pyodide.http import pyfetch
from ipywidgets import Dropdown, Button, HTML, FloatProgress, VBox, HBox

DATA_ROOT = "data_pkgs"
os.makedirs(DATA_ROOT, exist_ok=True)
CURRENT_FILE = os.path.join(DATA_ROOT, "CURRENT.txt")

# Configure your packages here:
packages = [
    {"name": "Sample (built-in, no download)", "id": "sample_local", "zip_url": None, "size_mb": 0},
    # {"name":"Set A (200 MB)","id":"setA","zip_url":"https://YOUR_BUCKET/setA.zip","size_mb":200},
    # {"name":"Set B (320 MB)","id":"setB","zip_url":"https://YOUR_BUCKET/setB.zip","size_mb":320},
]

dd = Dropdown(options=[(p["name"], p) for p in packages], description="Package:")
btn_dl = Button(description="Download", button_style="primary")
btn_use = Button(description="Use sample", button_style="")
btn_clear = Button(description="Delete current", button_style="danger")
msg = HTML("")
prog = FloatProgress(value=0.0, min=0, max=1, description="Download")

def set_current(pkg_id: str):
    with open(CURRENT_FILE, "w") as f:
        f.write(pkg_id)

async def download_and_unpack(url: str, outdir: str):
    head = await pyfetch(url, method="HEAD")
    total = int(head.headers.get("Content-Length", "0") or "0")
    r = await pyfetch(url)
    reader = r.body.getReader()
    rec = 0; chunks = []
    while True:
        part = await reader.read()
        if part.done: break
        b = bytes(part.value); chunks.append(b)
        rec += len(b)
        prog.value = (rec/total) if total else 0.0
    blob = b"".join(chunks)
    with zipfile.ZipFile(io.BytesIO(blob)) as zf:
        if os.path.isdir(outdir): shutil.rmtree(outdir)
        os.makedirs(outdir, exist_ok=True)
        zf.extractall(outdir)

def on_download(_):
    p = dd.value
    if p is None:
        msg.value = "<span style='color:red'>Pick a package first.</span>"
        return
    if p["zip_url"] is None:
        msg.value = "<b>Sample requires no download. Click 'Use sample'.</b>"
        return
    msg.value = f"Downloading <b>{p['name']}</b> (~{p['size_mb']} MB)…"
    prog.value = 0.0; prog.bar_style = ""
    async def run():
        try:
            await download_and_unpack(p["zip_url"], os.path.join(DATA_ROOT, p["id"]))
            set_current(p["id"])
            prog.value = 1.0; prog.bar_style = "success"
            msg.value = "<span style='color:green'>Package ready. Open <b>Dashboard.ipynb</b>.</span>"
        except Exception as e:
            prog.bar_style = "danger"
            msg.value = f"<span style='color:red'>Error: {e}</span>"
    asyncio.ensure_future(run())

def on_use(_):
    set_current("sample_local")
    msg.value = "<span style='color:green'>Using built-in sample. Open <b>Dashboard.ipynb</b>.</span>"

def on_clear(_):
    if os.path.isfile(CURRENT_FILE):
        try: os.remove(CURRENT_FILE)
        except Exception: pass
    msg.value = "Cleared current selection."
    prog.value = 0.0; prog.bar_style = ""

btn_dl.on_click(on_download)
btn_use.on_click(on_use)
btn_clear.on_click(on_clear)

VBox([dd, HBox([btn_dl, btn_use, btn_clear]), prog, msg])



**Next:** Open **`Dashboard.ipynb`** to explore filters and plots.  
You can return here anytime to switch packages.
